In [ ]:
# @title Environment setup
# Cell 1: Environment setup

import os

print("Cloning repository...")
!git clone https://github.com/MattyMroz/ColabUniversalDownloader.git

# Move into repo
os.chdir('ColabUniversalDownloader')
print(f"Working dir: {os.getcwd()}")

# Install dependencies from requirements.txt
print("\n🐍 Installing Python dependencies from requirements.txt...")
!pip install -q -r requirements.txt
print("✅ Python dependencies installed.")

print("\n\n🎉 Environment ready for tests!")

# Temporary directories
TMP_PIXELDRAIN: str = "./tmp_pixeldrain"  # @param {type:"string"}
TMP_MEGA: str = "./tmp_mega"  # @param {type:"string"}

# Test parameters
DELETE_DELAY_SECONDS: int = 60  # @param {type:"number"}
CLEANUP_TMP_AFTER_TESTS: bool = True  # @param {type:"boolean"}

# Test links (you can change)
PIXEL_URL: str = "https://pixeldrain.com/u/"  # @param {type:"string"}
MEGA_FILE_URL: str = "https://mega.nz/file/"  # @param {type:"string"}
MEGA_FOLDER_URL: str = "https://mega.nz/folder/"  # @param {type:"string"}

# Target drive and folder in Google Drive
TARGET_DRIVE: str = "my"  # @param ["my", "shared"]
SHARED_DRIVE_NAME: str = ""  # @param {type:"string"}
SESSION_FOLDER_NAME: str = "ColabUniversalDownloader"  # @param {type:"string"}

# Cell 2: Import modules and initialize tools (minimal, with optional progress)

# Standard library imports
import sys as _sys
import sys
from pprint import pprint

# Third-party imports
from IPython.display import clear_output  # type: ignore

# Local imports
from utils.google_drive import GoogleDriveManager
from utils.mega import MegaDownloader
from utils.pixeldrain import PixelDrainDownloader
from utils.progress import ConsoleProgress

# Allow importing utils when running standalone in Colab
sys.path.append('/content/ColabUniversalDownloader')

# Initialize classes
pixeldrain_handler: PixelDrainDownloader = PixelDrainDownloader()
mega_handler: MegaDownloader = MegaDownloader()
gdrive_handler: GoogleDriveManager = GoogleDriveManager()
progress_cb = ConsoleProgress()

print("✅ Modules imported; objects ready.")
print(f"Google Drive Manager ready: {gdrive_handler.is_ready()}")


import os
import re
import time
import shutil
import uuid
from pathlib import Path

# Unique session tag to avoid overwriting
SESSION_ID = time.strftime("%Y%m%d-%H%M%S") + "-" + uuid.uuid4().hex[:6]


def ensure_unique_dir(base_dir: str, label: str) -> str:
    """Return a path to a unique subdirectory in base_dir: base_dir/label-<SESSION_ID>."""
    p = Path(base_dir) / f"{label}-{SESSION_ID}"
    p.mkdir(parents=True, exist_ok=True)
    return str(p)


def safe_join(base_dir: str, filename: str) -> str:
    """Join directory and filename, preventing escaping above base_dir."""
    base = Path(base_dir).resolve()
    full = (base / filename).resolve()
    if base not in full.parents and base != full:
        # If the path escapes the base directory, fallback to base/filename.name
        full = base / Path(filename).name
    return str(full)


# Mega: filter messages to essentials (progress + summaries)
IMPORTANT_PATTERNS = [
    r"^\s*\d+%",                # progress lines
    r"Downloaded ",             # download completions
    r"Downloading ",            # download start
    r"^Error|^BŁĄD|^❌|^⚠️",   # errors/warnings
]
IMPORTANT_REGEX = re.compile("|".join(IMPORTANT_PATTERNS), re.IGNORECASE)


def mega_progress_minimal(line: str):
    if IMPORTANT_REGEX.search(line):
        print(line, end="")


# Pixeldrain: buffer lines and print once after finishing
class SingleLineLogger:
    def __init__(self):
        self._buf = []

    def __call__(self, line: str):
        # limit buffer size
        if len(self._buf) < 2000:
            self._buf.append(line.strip())

    def flush(self, header: str = None):
        if header:
            print(header)
        if self._buf:
            # show only the last line or a brief summary
            last = self._buf[-1]
            print(last)
        self._buf.clear()


single_line_logger = SingleLineLogger()

In [ ]:
# @title Test 1: Pixeldrain
# Cell 3: Test `pixeldrain.py` with unified progress

import os
import shutil
from typing import Optional

print("="*50)
print("🚀 TEST 1: PIXELDRAIN DOWNLOAD 🚀")
print("="*50)

# Temp folder (unique per session)
PXD_TMP_BASE: str = TMP_PIXELDRAIN
PXD_TMP: str = ensure_unique_dir(PXD_TMP_BASE, "pixeldrain")
os.makedirs(PXD_TMP, exist_ok=True)

filepath: Optional[str] = None
prev_cwd: str = os.getcwd()

try:
    os.chdir(PXD_TMP)
    from utils.pixeldrain import PixelDrainError
    filepath = pixeldrain_handler.download(PIXEL_URL, progress=progress_cb)

    if filepath and os.path.exists(filepath):
        print(f"\n✅ PIXELDRAIN OK: {filepath}")
        try:
            print(f"Size: {os.path.getsize(filepath) / 1024:.2f} KB")
        except Exception:
            pass
    else:
        print("\n❌ PIXELDRAIN DOWNLOAD FAILED")

except PixelDrainError as e:
    print(f"\n❌ PixelDrain error: {e}")
finally:
    try:
        os.chdir(prev_cwd)
    except Exception:
        pass
    if CLEANUP_TMP_AFTER_TESTS:
        shutil.rmtree(PXD_TMP, ignore_errors=True)

In [ ]:
# @title Test 2: Mega.nz (file + folder)
# Cell 4: Test `mega.py` — unified progress

import os
import shutil
from typing import List, Optional

print("="*50)
print("🚀 TEST 2: MEGA.NZ — PLIK I FOLDER 🚀")
print("="*50)

MEGA_TMP_BASE = TMP_MEGA
MEGA_TMP = ensure_unique_dir(MEGA_TMP_BASE, "mega")
os.makedirs(MEGA_TMP, exist_ok=True)

# --- A: Download single file ---
print("\n--- TEST 2A: Pobieranie pliku ---")
try:
    file_path: Optional[str] = mega_handler.download_file(
        MEGA_FILE_URL, dest_dir=MEGA_TMP, progress=progress_cb)
    if file_path and os.path.exists(file_path):
        print(f"✅ PLIK: {file_path}")
    else:
        print("❌ Nie udało się pobrać pliku (sprawdź URL).")
except Exception as e:
    print(f"❌ Błąd pobierania pliku: {e}")

# --- B: Download entire folder ---
print("\n--- TEST 2B: Pobieranie folderu (wszystko) ---")
try:
    results: List[str] = mega_handler.download_folder(
        MEGA_FOLDER_URL, dest_dir=MEGA_TMP, choose_files=False, progress=progress_cb)
    if results:
        print("✅ FOLDER — pliki wykryte:")
        for p in results:
            print(f" - {p}")
    else:
        # fallback: scan output directory
        collected: List[str] = []
        for root, _, files in os.walk(MEGA_TMP):
            for f in files:
                collected.append(os.path.join(root, f))
        if collected:
            print("ℹ️ Pliki znalezione na dysku:")
            for p in collected:
                print(f" - {p}")
        else:
            print("❌ Nie udało się pobrać folderu (sprawdź URL).")
except Exception as e:
    print(f"❌ Błąd pobierania folderu: {e}")

print("\n🎯 Testy Mega zakończone.")

if CLEANUP_TMP_AFTER_TESTS:
    shutil.rmtree(MEGA_TMP, ignore_errors=True)

In [ ]:
# @title Test 3: Google Drive (upload Mega file + Pixeldrain file)
# Cell 5: Test `google_drive.py` with progress

print("="*50)
print("🚀 TEST 3: GOOGLE DRIVE OPERATIONS 🚀")
print("="*50)
print("NOTE: This cell will open a Google auth window. Allow access.")

if not gdrive_handler.is_ready():
    print("\n❌ TEST FAILED. Google Drive Manager not properly initialized.")
else:
    # 1) Determine target drive and parent folder ID
    is_shared: bool = (TARGET_DRIVE == "shared")
    drive_name: str = SHARED_DRIVE_NAME if is_shared else "My Drive"
    parent_id: str | None = gdrive_handler.get_drive_id(
        drive_name=drive_name, is_shared=is_shared)
    if not parent_id:
        print("❌ Could not get target drive ID. Check shared drive name.")
    else:
        print(
            f"✅ Target drive: {'Shared' if is_shared else 'My Drive'} ({drive_name})")

        # 2) Ensure TMP directories for the session exist
        from pathlib import Path
        MEGA_TMP_BASE: str = TMP_MEGA
        PXD_TMP_BASE: str = TMP_PIXELDRAIN
        MEGA_TMP: str = ensure_unique_dir(MEGA_TMP_BASE, "mega")
        PXD_TMP: str = ensure_unique_dir(PXD_TMP_BASE, "pixeldrain")
        Path(MEGA_TMP).mkdir(parents=True, exist_ok=True)
        Path(PXD_TMP).mkdir(parents=True, exist_ok=True)

        # 3) Download files (only Mega file and one Pixeldrain file)
        print("\n➡️ Downloading from Mega (file)...")
        mega_file_path: str | None = mega_handler.download_file(
            MEGA_FILE_URL, dest_dir=MEGA_TMP, progress=progress_cb)

        print("➡️ Downloading from Pixeldrain…")
        import os as _os
        prev: str = _os.getcwd()
        _os.chdir(PXD_TMP)
        pix_file: str | None = pixeldrain_handler.download(
            PIXEL_URL, progress=progress_cb)
        _os.chdir(prev)

        # 4) Collect files to upload (just the two files if present)
        import os
        files_to_upload: list[str] = []
        if mega_file_path and os.path.exists(mega_file_path):
            files_to_upload.append(mega_file_path)
        if pix_file and os.path.exists(pix_file):
            files_to_upload.append(pix_file)
        print("\n📦 Files to upload:")
        for p in files_to_upload:
            print(f" - {p}")

        if not files_to_upload:
            print("\n❌ No files to upload.")
        else:
            # 5) Create session folder on Drive
            print("\n➡️ Creating session folder on Google Drive...")
            from googleapiclient.errors import HttpError
            try:
                folder_metadata: dict[str, object] = {
                    'name': SESSION_FOLDER_NAME,
                    'mimeType': 'application/vnd.google-apps.folder',
                    'parents': [parent_id]
                }
                folder = gdrive_handler.drive_service.files().create(
                    body=folder_metadata,
                    fields='id, webViewLink',
                    supportsAllDrives=True
                ).execute()
                session_folder_id: str = folder['id']
                session_folder_link: str | None = folder.get('webViewLink')
                print(
                    f"✅ Folder created: {SESSION_FOLDER_NAME} (ID: {session_folder_id})")
                print(f"🔗 Link: {session_folder_link}")
            except HttpError as he:
                print(f"⚠️ Folder exists or error: {he}")
                q: str = f"name = '{SESSION_FOLDER_NAME}' and mimeType = 'application/vnd.google-apps.folder' and '{parent_id}' in parents and trashed = false"
                resp = gdrive_handler.drive_service.files().list(q=q, fields='files(id, webViewLink)',
                                                                 supportsAllDrives=True, includeItemsFromAllDrives=True).execute()
                items: list[dict[str, object]] = resp.get('files', [])
                if items:
                    session_folder_id = items[0]['id']  # type: ignore[index]
                    session_folder_link = items[0].get(
                        'webViewLink')  # type: ignore[index]
                    print(
                        f"ℹ️ Using existing folder: {SESSION_FOLDER_NAME} (ID: {session_folder_id})")
                    print(f"🔗 Link: {session_folder_link}")
                else:
                    raise

            # 6) Upload files + public links (with progress callback)
            print("\n➡️ Uploading files to Google Drive and sharing (with progress)…")
            uploaded: list[dict[str, object]] = []
            for p in files_to_upload:
                info: dict[str, object] | None = gdrive_handler.upload_and_share(
                    p, session_folder_id, skip_if_exists=True, replace_if_exists=False, progress=progress_cb)
                if info and info.get('id'):
                    if info.get('link'):
                        print(f"✅ Upload: {os.path.basename(p)}")
                        print(f"   🔗 Link: {info['link']}")
                    else:
                        print(
                            f"✅ Upload: {os.path.basename(p)} (no public link)")
                    uploaded.append(info)
                else:
                    print(f"❌ Upload failed: {p}")

            # 7) Delete session folder now or schedule
            delay: int = int(DELETE_DELAY_SECONDS)
            try:
                if delay <= 0:
                    print("\n🧹 Deleting session folder now (after cleaning contents):")
                    print(f" - {session_folder_id}")
                    gdrive_handler.delete_folder_now(session_folder_id)
                else:
                    print(
                        f"\n🧹 Scheduled delete for folder: {session_folder_id} in {delay}s")
                    gdrive_handler.delete_folder_after_delay(
                        session_folder_id, delay_seconds=delay)
            except Exception:
                pass

            # 8) Links summary
            print("\n📁 Summary:")
            print(f"Folder: {SESSION_FOLDER_NAME}")
            if session_folder_link:
                print(f"🔗 Browse folder: {session_folder_link}")
            else:
                base_drive: str = "https://drive.google.com/drive/"
                print(
                    f"🔗 Drive: {base_drive}{'shared-drives' if is_shared else 'my-drive'}")

        # 9) Cleanup
        if CLEANUP_TMP_AFTER_TESTS:
            import shutil as _sh
            _sh.rmtree(MEGA_TMP, ignore_errors=True)
            _sh.rmtree(PXD_TMP, ignore_errors=True)